In [57]:
%config IPCompleter.greedy=True
import re
import json
import csv
from collections import defaultdict, Counter
from operator import itemgetter
from tqdm import tqdm_notebook as tqdm
from time import time
import pandas as pd
import numpy as np

In [48]:
def json_read(filename):
    with open(filename, 'r') as inf:
        res = json.load(inf)
    return res

def json_dump(obj, filename, ea=False, indent=4):
    with open(filename, 'w') as ouf:
        json.dump(obj, ouf, ensure_ascii=ea, indent=indent)

In [49]:
def get_question_task_map(filename):
    res = {}
    data = json_read(filename)
    for entry in data:
        res[entry['question']] = entry
    return res

In [50]:
def get_answer_ids_from_json(json_str):
    anses = json.loads(json_str)
    res = []
    for k, v in anses.items():
        if v:
            res.append(int(k))
    return tuple(res)

## Analyze questions

### Learning tasks

In [5]:
q_task_map = get_question_task_map('q_matches_1_5000.json')

In [10]:
results = pd.read_csv('questions/learning_task_results.tsv', sep='\t', header=0)
results.dropna(how='all', inplace=True)

In [11]:
results.head()

,INPUT:answer,INPUT:matches,INPUT:question,OUTPUT:q_matches,GOLDEN:q_matches,HINT:text,ASSIGNMENT:link,ASSIGNMENT:assignment_id,ASSIGNMENT:worker_id,ASSIGNMENT:status,ASSIGNMENT:started
0,Телевизор,"{""text"":""Меню «Пуск»""\,""link_text"":""Википедия""...","Что, по мнению почтальона Печкина, является гл...","{""1"":true,""2"":false,""3"":false,""4"":false,""5"":fa...","{""1"":false,""2"":false,""3"":false,""4"":false,""5"":f...","Нужно выбрать вариант 8, так как он соответств...",https://toloka.yandex.ru/task/9493996/000090dd...,000090ddec--5e00cc63f5a076011e42651e,f2ca3d107543223122d74437faf968dd,APPROVED,2019-12-23T14:17:07.412
1,Такова жизнь,"{""text"":""C’est la vie""\,""link_text"":""Википедия...","Как переводится с французского фраза: ""C est l...","{""1"":true,""2"":false,""3"":false,""4"":false,""5"":fa...","{""1"":false,""2"":false,""3"":false,""4"":false,""5"":f...",Ни один из вариантов не подходит. Варианты 1 и...,https://toloka.yandex.ru/task/9493996/000090dd...,000090ddec--5e00cc63f5a076011e42651e,f2ca3d107543223122d74437faf968dd,APPROVED,2019-12-23T14:17:07.412
2,Перчатку,"{""text"":""Братья-сёстры\, соперники-соперницы""\...","Что бросают, вызывая соперника на дуэль?","{""1"":true,""2"":false,""3"":false,""4"":false,""5"":fa...","{""1"":false,""2"":false,""3"":true,""4"":false,""5"":fa...","Подходит только вариант 3, варианты 4-8 соотве...",https://toloka.yandex.ru/task/9493996/000090dd...,000090ddec--5e00cc63f5a076011e42651e,f2ca3d107543223122d74437faf968dd,APPROVED,2019-12-23T14:17:07.412
3,"Детское Село, в 1937 - Пушкин","{""text"":""Пушкин (город в составе Санкт-Петербу...",Какое название в 1918 году дали городу Царское...,"{""1"":true,""2"":false,""3"":false,""4"":false,""5"":fa...","{""1"":true,""2"":false,""3"":false,""4"":false,""5"":fa...","Несмотря на то, что в вопросе город упоминаетс...",https://toloka.yandex.ru/task/9493996/000090dd...,000090ddec--5e00cc63f5a076011e42651e,f2ca3d107543223122d74437faf968dd,APPROVED,2019-12-23T14:17:07.412
5,Телевизор,"{""text"":""Меню «Пуск»""\,""link_text"":""Википедия""...","Что, по мнению почтальона Печкина, является гл...","{""1"":false,""2"":false,""3"":false,""4"":false,""5"":f...","{""1"":false,""2"":false,""3"":false,""4"":false,""5"":f...","Нужно выбрать вариант 8, так как он соответств...",https://toloka.yandex.ru/task/9493996/000090dd...,000090ddec--5e00cc68ba4166012152ab1f,7e5876f94c46e10ad540a07456cbee37,APPROVED,2019-12-23T14:17:12.510


In [41]:
stats_by_question = {}

In [42]:
for index, row in results.iterrows():
    q = row['INPUT:question']
    if q not in stats_by_question:
        stats_by_question[q] = defaultdict(int)
    ans = get_answer_ids_from_json(row['OUTPUT:q_matches'])
    stats_by_question[q][ans] += 1

In [43]:
for q in stats_by_question:
    anses = stats_by_question[q]
    anses = [(ans, f) for ans, f in anses.items()]
    anses.sort(key=itemgetter(1), reverse=True)
    stats_by_question[q] = anses

In [49]:
with open('questions/learning_task_stats.txt', 'w') as ouf:
    for q, stats in stats_by_question.items():
        task = q_task_map[q]
        print(q, file=ouf)
        for i, match in enumerate(task['matches'], 1):
            name = match['name']
            if match['ruwiki']:
                page = match['ruwiki'].replace(' ', '_')
                url = f'https://ru.wikipedia.org/wiki/{page}'
            else:
                page = match['qid']
                url = f'https://www.wikidata.org/wiki/{page}'
            print(i, name, file=ouf)
            print(url, file=ouf)
        print(file=ouf)
        for ans, f in stats:
            print(ans, f'count: {f}', file=ouf)
        print('\n', file=ouf)

### Labeling

In [51]:
q_task_map = get_question_task_map('questions/q_matches_1_5000.json')

In [52]:
assignments = pd.read_csv('questions/assignments_501_1000.tsv', sep='\t', header=0)
assignments.dropna(how='all', inplace=True)

In [53]:
assignments.head(10)

,INPUT:answer,INPUT:matches,INPUT:question,OUTPUT:q_matches,GOLDEN:q_matches,HINT:text,ASSIGNMENT:link,ASSIGNMENT:assignment_id,ASSIGNMENT:worker_id
0,Половцы,"{""url"":""https://ru.wikipedia.org/wiki/Игорь_Рю...",Кто пленил князя Игоря?,"{""1"":true,""2"":false,""3"":false,""4"":false,""5"":fa...",NaN,NaN,https://toloka.yandex.ru/task/9495159/000090e2...,000090e277--5e00cf8f2f3cb8011d3412a2,44d034a4c4a3a986bb263d4d4bf32af6
1,После чемпионата Квиддича,"{""url"":""https://ru.wikipedia.org/wiki/Гарри\,_...",Где Гарри впервые поцеловал Джинни?,"{""1"":true,""2"":false,""3"":false,""4"":false,""5"":fa...",NaN,NaN,https://toloka.yandex.ru/task/9495159/000090e2...,000090e277--5e00cf8f2f3cb8011d3412a2,44d034a4c4a3a986bb263d4d4bf32af6
2,«Дубровский»,"{""url"":""https://ru.wikipedia.org/wiki/Памятник...",Какая повесть есть у А.С. Пушкина?,"{""1"":false,""2"":false,""3"":false,""4"":false,""5"":t...",NaN,NaN,https://toloka.yandex.ru/task/9495159/000090e2...,000090e277--5e00cf8f2f3cb8011d3412a2,44d034a4c4a3a986bb263d4d4bf32af6
3,Пирогами,"{""url"":""https://ru.wikipedia.org/wiki/Красная_...","Чем, согласно русской пословице, красна изба?","{""1"":false,""2"":false,""3"":false,""4"":false,""5"":f...",NaN,NaN,https://toloka.yandex.ru/task/9495159/000090e2...,000090e277--5e00cf8f2f3cb8011d3412a2,44d034a4c4a3a986bb263d4d4bf32af6
4,Джон Булль,"{""url"":""https://ru.wikipedia.org/wiki/Елизавет...",Как звали придворного органиста английской кор...,"{""1"":true,""2"":false,""3"":false,""4"":false,""5"":fa...",NaN,NaN,https://toloka.yandex.ru/task/9495159/000090e2...,000090e277--5e00cf8f2f3cb8011d3412a2,44d034a4c4a3a986bb263d4d4bf32af6
5,Манхэттен,"{""url"":""https://ru.wikipedia.org/wiki/Президен...",Какой район Нью-Йорка находится на острове?,"{""1"":false,""2"":true,""3"":false,""4"":false,""5"":fa...","{""1"":false,""2"":true,""3"":false,""4"":false,""5"":fa...",NaN,https://toloka.yandex.ru/task/9495159/000090e2...,000090e277--5e00cf8f2f3cb8011d3412a2,44d034a4c4a3a986bb263d4d4bf32af6
6,Хорезмскую,"{""url"":""https://ru.wikipedia.org/wiki/Голод_в_...",Какую республику упразнили в СССР в 1924г.?,"{""1"":false,""2"":false,""3"":true,""4"":false,""5"":fa...",NaN,NaN,https://toloka.yandex.ru/task/9495159/000090e2...,000090e277--5e00cf8f2f3cb8011d3412a2,44d034a4c4a3a986bb263d4d4bf32af6
7,Россия,"{""url"":""https://ru.wikipedia.org/wiki/Озеро_ст...",В какой стране больше всего рек и озёр?,"{""1"":false,""2"":false,""3"":false,""4"":false,""5"":f...",NaN,NaN,https://toloka.yandex.ru/task/9495159/000090e2...,000090e277--5e00cf8f2f3cb8011d3412a2,44d034a4c4a3a986bb263d4d4bf32af6
8,Голландия,"{""url"":""https://ru.wikipedia.org/wiki/Нидерлан...",Как иначе называют Нидерланды?,"{""1"":true,""2"":false,""3"":false,""4"":false,""5"":fa...",NaN,NaN,https://toloka.yandex.ru/task/9495159/000090e2...,000090e277--5e00cf8f2f3cb8011d3412a2,44d034a4c4a3a986bb263d4d4bf32af6
9,Малиновое,"{""url"":""https://ru.wikipedia.org/wiki/Альбус_Д...",Какое любимое варенье Альбуса Дамблдора?,"{""1"":true,""2"":false,""3"":false,""4"":false,""5"":fa...",NaN,NaN,https://toloka.yandex.ru/task/9495159/000090e2...,000090e277--5e00cf8f2f3cb8011d3412a2,44d034a4c4a3a986bb263d4d4bf32af6


In [54]:
responses = defaultdict(list)
for index, row in assignments.iterrows():
    q = row['INPUT:question']
    ans = get_answer_ids_from_json(row['OUTPUT:q_matches'])
    responses[q].append(ans)

In [69]:
with open('questions/assign_501_1000_stats.txt', 'w') as ouf:
    for q, anses in responses.items():
        task = q_task_map[q]
        print(q, file=ouf)
        for i, match in enumerate(task['matches'], 1):
            name = match['name']
            if match['ruwiki']:
                page = match['ruwiki'].replace(' ', '_')
                url = f'https://ru.wikipedia.org/wiki/{page}'
            else:
                page = match['qid']
                url = f'https://www.wikidata.org/wiki/{page}'
            print(i, name, file=ouf)
            print(url, file=ouf)
        print(file=ouf)
        for ans in anses:
            print(ans, file=ouf)
        print(file=ouf)

In [55]:
responses

defaultdict(list,
            {'Кто пленил князя Игоря?': [(1,), (1,), (1,), (3,), (1,)],
             'Где Гарри впервые поцеловал Джинни?': [(1,), (), (1,), (), ()],
             'Какая повесть есть у А.С. Пушкина?': [(5,),
              (5,),
              (5,),
              (5,),
              (5,)],
             'Чем, согласно русской пословице, красна изба?': [(7,),
              (),
              (),
              (4,),
              (4,)],
             'Как звали придворного органиста английской королевы Елизаветы – по преданию, автора английского гимна?': [(1,),
              (),
              (),
              (),
              ()],
             'Какой район Нью-Йорка находится на острове?': [(2,),
              (),
              (2,),
              (2,),
              (2, 3),
              (),
              (2,),
              (2,),
              (2, 3),
              (2,),
              (1,),
              (2,),
              (2,),
              (2,),
              (2,),
 

In [74]:
annotation = {}
for q, v in responses.items():
    markers = Counter([i - 1 for t in v for i in t])
    n = len(v)
    annotation[q] = []
    for mark, cnt in markers.items():
        if cnt >= n / 2:
            annotation[q].append(mark)

## Analyze answers

### Save labeling to file

In [167]:
q_task_map = get_question_task_map('answers/all_matches.json')

In [218]:
part = '5001_10000'

In [234]:
assignments = pd.read_csv(f'answers/aggregated_results_{part}.tsv', sep='\t', header=0, quotechar='|')
assignments.dropna(how='all', inplace=True)

In [250]:
cnt = 0
for part in ['500_5000', '5001_10000', '10001_14000', '14001_end']:
    assignments = pd.read_csv(f'answers/aggregated_results_{part}.tsv', sep='\t', header=0, quotechar='|')
    assignments.dropna(how='all', inplace=True)
    cnt += len(assignments)
print(cnt)

14435


In [3]:
aggs = []
aggs = [
    pd.read_csv('answers/aggregated_results_500_5000.tsv', sep='\t', header=0, quotechar='|'),
    pd.read_csv('answers/aggregated_results_5001_10000.tsv', sep='\t', header=0, quotechar='|'),
    pd.read_csv('answers/aggregated_results_10001_14000.tsv', sep='\t', header=0, quotechar='|'),
    pd.read_csv('answers/aggregated_results_14001_end.tsv', sep='\t', header=0, quotechar='|')
]
agg = pd.concat(aggs)

In [7]:
agg[agg['OUTPUT:match'] == 0].to_csv('answers/not_matched_answers.csv')

In [170]:
with open(f'answers/labeling_result_pretty_{part}.txt', 'w') as ouf:
    for index, row in assignments.iterrows():
        q = row['INPUT:question']
        a = row['INPUT:answer']
        print(q, file=ouf)
        print(a, file=ouf)
        
        for i, match in enumerate(q_task_map[q]['matches'], 1):
            name = match['name']
            if match['ruwiki']:
                page = match['ruwiki'].replace(' ', '_')
                url = f'https://ru.wikipedia.org/wiki/{page}'
            else:
                page = match['qid']
                url = f'https://www.wikidata.org/wiki/{page}'
            print(i, name, file=ouf)
            print(url, file=ouf)
        print(file=ouf)
        print(row['OUTPUT:match'], row['CONFIDENCE:match'], file=ouf)
        print(file=ouf)

In [171]:
q_ans_entity_map = {}

for index, row in assignments.iterrows():
    q = row['INPUT:question']
    matches = q_task_map[q]['matches']
    match_id = row['OUTPUT:match'] - 1
    if match_id >= 0:
        q_ans_entity_map[q] = matches[match_id]['qid']
        
json_dump(q_ans_entity_map, f'answers/answers_{part}_labeling.json')

In [173]:
q_ans_entity_map = {}
for part in ['500_5000', '5001_10000', '10001_14000', '14001_end']:
    qam = json_read(f'answers/answers_{part}_labeling.json')
    q_ans_entity_map.update(qam)

In [253]:
len(q_ans_entity_map)

9655

In [252]:
json_dump(q_ans_entity_map, 'answers/answers_labeling.json')

In [107]:
s = 0
c = 0
for i, v in agg['OUTPUT:match'].value_counts().sort_index()[1:].iteritems():
    c += v * (1 / i)
    s += v
print(c / s)

0.8516354751181268


In [108]:
s = 0
c = 0
for i, v in agg['OUTPUT:match'].value_counts().sort_index()[1:].iteritems():
    c += v * i
    s += v
print(c / s)

1.5013982392542724


In [37]:
agg.sort_index()

,SYSTEM:task_id,INPUT:answer,INPUT:matches,INPUT:question,OUTPUT:match,CONFIDENCE:match,GOLDEN:match,Unnamed: 7
0,00007b963a--5dcae89b4d462b0118848f41,Христофор Колумб,"[{""url"":""https://ru.wikipedia.org/wiki/Колумб,...","Кто собирался открыть Индию, а открыл Америку?",1,99.99%,NaN,NaN
0,0000807cb0--5dcc72134d462b0118929ff7,24,"[{""url"":""https://ru.wikipedia.org/wiki/24_(чис...","Сколько рисунков, как правило, нужно нарисоват...",1,99.99%,NaN,NaN
0,0000807ad1--5dcc6a214d462b0118926956,майор,"[{""url"":""https://ru.wikipedia.org/wiki/Майор"",...",В каком звании был следователь Знаменский при ...,1,99.99%,NaN,NaN
0,00008077ea--5dcc63f2e48f45011a3f94bc,Лишённый наследства,"[{""url"":""https://ru.wikipedia.org/wiki/Наследо...","Какой девиз был выбит на щите рыцаря Айвенго, ...",0,99.52%,NaN,NaN
1,0000807cb0--5dcc72134d462b0118929ff8,Надин арап,"[{""url"":""https://ru.wikipedia.org/wiki/Надино""...",Какая фраза является анаграммой слова ПАНДАРИНА?,0,99.99%,NaN,NaN
1,00008077ea--5dcc63f2e48f45011a3f94bd,Прыжки в длинну,"[{""url"":""https://ru.wikipedia.org/wiki/Прыжок_...",В какой легкоатлетической дисциплине олимпийск...,1,99.99%,NaN,NaN
1,00007b963a--5dcae89b4d462b0118848f42,Шиповник,"[{""url"":""https://ru.wikipedia.org/wiki/Шиповни...","Какое растение называют ""дикой розой""?",1,100.00%,1.0,NaN
1,0000807ad1--5dcc6a214d462b0118926957,цзягувэнь,"[{""url"":""https://ru.wikipedia.org/wiki/Цзягувэ...",Как называются иероглифические надписи на гада...,1,99.99%,NaN,NaN
2,00008077ea--5dcc63f2e48f45011a3f94be,Футбол,"[{""url"":""https://ru.wikipedia.org/wiki/Футбол""...",В каком виде спорта успехов добились учёный Ни...,1,99.99%,NaN,NaN
2,0000807cb0--5dcc72134d462b0118929ff9,Томас Грэм,"[{""url"":""https://ru.wikipedia.org/wiki/Грэм,_Т...",Кто является первооткрывателем процесса диализа?,1,99.95%,NaN,NaN


In [40]:
n_candidates = []
for i, matches in agg['INPUT:matches'].iteritems():
    matches = json.loads(matches)
    n_candidates.append(len(matches))
n_candidates = np.array(n_candidates)

In [42]:
n_candidates.mean()

5.439140976792518

In [44]:
np.median(n_candidates)

5.0

In [126]:
aggs = []
aggs = [
    pd.read_csv('answers/aggregated_results_500_5000.tsv', sep='\t', header=0, quotechar='|', keep_default_na=False),
    pd.read_csv('answers/aggregated_results_5001_10000.tsv', sep='\t', header=0, quotechar='|', keep_default_na=False),
    pd.read_csv('answers/aggregated_results_10001_14000.tsv', sep='\t', header=0, quotechar='|', keep_default_na=False),
    pd.read_csv('answers/aggregated_results_14001_end.tsv', sep='\t', header=0, quotechar='|', keep_default_na=False)
]
agg = pd.concat(aggs)

In [88]:
confs = np.array([float(p.strip('%')) for p in agg['CONFIDENCE:match'].values])

In [91]:
print(f'Average confidence is {confs.mean()}')
print(f'Median confidence is {np.median(confs)}')

Average confidence is 98.57530723934879
Median confidence is 99.98


In [106]:
print(f'Number of tasks with confidence less than 85 is {sum(confs < 85)}')

Number of tasks with confidence less than 85 is 264


In [136]:
cnt = 0
cnt_m = 0
for i, row in agg.iterrows():
    ans = row['INPUT:answer']
    if re.search(r'\d', ans):
        q = row['INPUT:question']
        
        if row['OUTPUT:match'] != 0:
            cnt_m += 1
            print(ans, q, sep='\t')
        cnt += 1
print(cnt)
print(cnt_m)

В середине 17 века (в 1662 г.)	Когда основан город Курган?
1 мая	Какого числа отмечают День солидарности трудящихся?
формула-1	Какие автомобили получили свое название от названия списка требований к их техническим данным?
2 сентября	Какого числа закончилась Вторая Мировая Война? (назовите число и месяц)
7 ноября	Чем пpославился день 17 бpюмеpа? Это ...
1 килограмм	Сколько весят 2151071428570000000000000 атомов кремния?
14 февраля	Какого числа отмечают день Святого Валентина?
3000 м	Какой дистанции в беге на коньках для мужчин на зимних Олимпийских играх нет?
180 градусов	Чему равна сумма углов равностороннего треугольника?
100 монет	Сколько готова была заплатить Беладонна за поимку Фунтика?
360 градусов	Чему равна сумма углов выпуклого четерёхугольника?
11 ноября 1918	Когда окончилась Первая Мировая война?
45 размера покупал он сапоги	Какой размер обуви был у дяди Степы?
15	Сколько примерно преподавателей в Хогвартсе?
3	Сколько атомов кислорода содержится в молекуле озона?
571	В каком 

Гранатовый сок — 30 мл (сироп сахарный - 10 мл, игристое сладкое вино - 120 мл)	Какой сок входит в состав коктейля тинторетто?
300.	"Сколько лет исполнилось Санкт-Петербургу в 2003 году?"
2105.	Какая из указанных моделей "Жигулей" выпущена последней?
2	Сколько детей было у Анны Карениной?
3	Сколько раз надо сложить лист бумаги пополам, чтобы получить 1/8 его часть?
«Вокруг света за 80 дней»	В каком фильме 1956 года по роману Жюля Верна было задействовано рекордное число животных — 8552?
Ту-134	Какой пассажирский самолет имеет два двигателя ?
Алерт, на острове Элсмир территории Нунавут, Канада, всего в 817 километрах от Северного полюса.	Какое поселение в мире самое северное?
UTC+4	В каком часовом поясе находится город Москва?
1 апреля	Какого числа была образована корпорация Apple Computer?
"Моцарт и Сальери", зимой 1832 г.	Какая пьеса Пушкина была поставлена на сцене при жизни поэта?
2400	Сколько квадратных саженей в одной казенной десятине?
1520 мм	Какова ширина железнодорожной колеи 

1914	В каком году началась первая мировая война?
1147	В каком году основана Москва?
400	Сколько деревьев среднего размера расходует на свои нужды человек в течение жизни?
75	В скольких странах должен получить распространение мужской вид спорта, чтобы попасть на летнюю Олимпиаду?
1185	В каком году состоялся неудачный поход Игоря Святославича против половцев, послуживший темой "Слова о полку Игореве"?
8	Каким по счету месяцем года был октябрь до реформы календаря при Юлии Цезаре?
10	Сколько негритят было в известном произведении Агаты Кристи?
1912	В каком году затонул легендарный "Титаник"?
1799	В каком году родился писатель А.С. Пушкин
4	Сколько клапанов в сердце человека?
13	Сколько полос на американском флаге?
60	Сколько градусов составляет угол в равностороннем треугольнике?
32	Сколькими картами играют в преферанс?
36	Сколько карт необходимо для игры в "Дурака"?
2	Сколько букв «потерялось» в первоначальном названии яхты капитана Врунгеля?
1947	Когда были найдены "Свитки Мёртвого моря

500	Как записать число 1280 в шестнадцатеричной системе счисления?
1	Сколько звезд на погоне генерала-майора российской армии?
2500-летия основания Персидской империи	Какую памятную медаль учредили в Иране в 1971 году?
1944	В каком году был построен Керченский железнодорожный мост, соединявший Крым с Таманским полуостровом?
4	Сколько законов роботехники предложил Айзек Азимов?
6	Сколько человек изображено на картине Н. Ярошенко "Всюду жизнь"?
1 конфета	Какая стоимость была установлена Карлсоном для входа на «Вечер чудес»?
16	Какое максимальное число промахов может совершить спортсмен за один этап биатлонной эстафеты?
24	Сколько рисунков, как правило, нужно нарисовать для одной секунды мультфильма?
в году 10 месяцев	Какое утверждение неверно для десятичной системы измерения времени, принятой в Первой французской республике?
-40 градусов Цельсия	Какова приблизительная средняя температура на Северном полюсе зимой?
4	Сколько внутренних углов у ромба?
2	С какой цифры начинаются штрих коды, 